## Imports

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from coppeliasim_zmqremoteapi_client import *

WORLDX, WORLDY = 24, 24 

## Remote API Functions

In [12]:
# create a client to connect to zmqRemoteApi server:
# (creation arguments can specify different host/port,
# defaults are host='localhost', port=23000)
client = RemoteAPIClient()

# get a remote object:
sim = client.require('sim')

# call API function fo test:
pioneer = sim.getObject('/Pioneer_p3dx')
print("Printando o pioninho: " + str(pioneer))

Printando o pioninho: 13


In [13]:
# Função para rotacionar em torno do eixo z
def Rz(theta):
    return np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ])

def get_object_position(sim, object_name):
    return sim.getObjectPosition(sim.getObject(object_name), -1)

def get_object_orientation(sim, object_name):
    return sim.getObjectOrientation(sim.getObject(object_name), -1)

In [14]:
def readSensorData(range_data_signal_id="hokuyo_range_data", 
                    angle_data_signal_id="hokuyo_angle_data"):
    
    string_range_data = sim.getStringSignal(range_data_signal_id)

    string_angle_data = sim.getStringSignal(angle_data_signal_id)

    # verifique se ambos os dados foram obtidos corretamente
    if string_range_data != None and string_angle_data != None:
        # descompacte dados de mensagens de alcance e sensor
        raw_range_data = sim.unpackFloatTable(string_range_data)
        raw_angle_data = sim.unpackFloatTable(string_angle_data)

        return raw_range_data, raw_angle_data

    return None

# Função para desenhar os dados do laser
def draw_laser_data(laser_data, max_sensor_range=5):
    fig = plt.figure(figsize=(6,6), dpi=100)
    ax = fig.add_subplot(111, aspect='equal')

    # for ang, dist in laser_data:
    for i in range(len(laser_data)):
        ang, dist = laser_data[i]
        # Quando o feixe não acerta nada, retorna o valor máximo (definido na simulação)
        # Logo, usar um pequeno limiar do máximo para considerar a leitura
        if (max_sensor_range - dist) > 0.1:
            x = dist * np.cos(ang)
            y = dist * np.sin(ang)
            c = 'r'
            if ang < 0:    
                c = 'b'
            ax.plot(x, y, 'o', color=c)

    ax.plot(0, 0, 'k>', markersize=10)
        
    ax.grid()
    ax.set_xlim([-max_sensor_range, max_sensor_range])
    ax.set_ylim([-max_sensor_range, max_sensor_range])

## Função auxiliar para realizar uma transformacao para o referecial A

In [15]:
# Utiliza as posições e orientações de de um ponto em determinado referencial global e realiza a transformação homogênia
# para o referencial da posição e orientação de posA e  oriA

def transformacao(posA, oriA):
    # Criação da matriz de transformação homogênea
    RWA = Rz(oriA[2])

    TWA = np.array([[posA[0]], [posA[1]], [posA[2]]]) # Transforma a posição em um array

    # Adiciona uma linha [0, 0, 0, 1] à direita da matriz RWA
    HWA = np.column_stack((RWA, TWA))
    HWA = np.row_stack((HWA, [0, 0, 0, 1]))  # Adiciona a linha [0, 0, 0, 1] ao final da matriz HWA

    return HWA

## Força de Atração

In [16]:
def att_force(q, goal, katt= 0.1):
  Fatt = katt *(goal - q)
  return Fatt

## Força Total

In [17]:
def rep_force(q, obs, R=2, krep=.1):
    Frep = np.zeros(2)
    for obstacle in obs:
        v = q[0:2] - obstacle
        d = np.linalg.norm(v) 

        # Se a distância for menor que o raio de influência	calcular a força repulsiva
        if (d < R):  
            rep = (1/d**2)*((1/d)-(1/R))*(v/d) 
            Frep += rep

    return krep*Frep


## Força Total

In [18]:
def tt_force(q,goal,laser_data,obs,obs_pts,HWL, max_sensor_range = 5):
    Frep = np.zeros(2)
    for i in range(len(laser_data)):
        ang, dist = laser_data[i] #pega os valores de angulo e distância

        if (max_sensor_range - dist) > 0.1:
            x = dist * np.cos(ang) #meu x
            y = dist * np.sin(ang) #meu y
            point = np.array([x,y,0,1])

            if len(HWL) != 0:
                point = HWL @ point
                obs.append(point[0:2])
                obs_pts.append(point)  
    Frep = rep_force(q, obs)
    Fatt= att_force(q, goal)
    Ft = Fatt + Frep
    print("Ft new -> ",Ft)
            
    return Ft


## Leitura do sensor

In [20]:
# Conectando-se ao CoppeliaSim
# Run a simulation in asynchronous mode:
clientID = sim.startSimulation()

if clientID != -1:
    print("Connected to remote API server")

    # Handle para o ROBÔ    
    laser_robot = sim.getObject('/Pioneer_p3dx')

    #Handle para o LASER
    laser = sim.getObject('/Pioneer_p3dx/fastHokuyo')
    print("laser handle -> ", laser)

    # Handle para as juntas das RODAS
    motorLeft = sim.getObject('/Pioneer_p3dx_leftMotor')
    motorRight = sim.getObject('/Pioneer_p3dx_rightMotor')

    # Handle utilizado como GOAL 
    #goal = get_object_position(sim, '/tree') Utilizado durante o desenvolvimento para facilitar testes em diferentes posições

    # Handle do obstáculo
    obs = get_object_position(sim, '/muro')
    obs_ori = get_object_orientation(sim, '/muro')
    obs = np.array(obs[:2])
    obs = np.concatenate((obs, obs_ori))#[2:3]

    L = 0.381 # Distância entre as rodas
    r = 0.0975 # Raio da roda

    seguindo = False

    maxv = 1 # para limitar a velocidade linear
    maxw = np.deg2rad(45) # para limitar a velocidade angular

    Ft_x = 0
    Ft_y = 0

    while True:

        laser_robot_position = get_object_position(sim, '/Pioneer_p3dx')
        print("Posição do robô: ", laser_robot_position)

        laser_robot_orientation = get_object_orientation(sim, '/Pioneer_p3dx')
        
        # Handle para os dados do LASER
        laser_range_data = "hokuyo_range_data"
        laser_angle_data = "hokuyo_angle_data"
            
        if laser_range_data is not None and laser_angle_data is not None:
            raw_range_data, raw_angle_data = readSensorData() 

        laser_data = np.array([raw_angle_data, raw_range_data]).T
        laser_pos = get_object_position(sim, '/Pioneer_p3dx/fastHokuyo')
        laser_ori = get_object_orientation(sim, '/Pioneer_p3dx/fastHokuyo')
        #print("Laser pos -> ", laser_pos)
        #print("Laser ori -> ", laser_ori)

        Hlw = transformacao(laser_pos, laser_ori) # Matriz de transformação do laser para o mundo

        v = 0.4 # Velocidade linear
        w = 0 # Velocidade angular
        
        q = [laser_robot_position[0], laser_robot_position[1]] # Posição do robô
        #goal = goal[:2] # Posição do goal
        goal = [0.5, 4.75]
        laser_robot_position2d = laser_robot_position[:2] # Posição do robô/laser em 2D
        obs = [] # Lista de obstáculos
        obs_pts = [] # Lista de pontos dos obstáculos

        Ft = tt_force(np.array(laser_robot_position2d), np.array(goal), laser_data, obs, obs_pts, Hlw)

        print("Força total -> ", Ft)

        Ft_x = Ft[0]
        Ft_y = Ft[1]

        # Constantes 
        kv = 1
        kw = 2

        # Verificar se o robô está de costas para o objetivo
        robot_to_goal_angle = np.arctan2(goal[1] - laser_robot_position[1], goal[0] - laser_robot_position[0])
        angle_difference = (robot_to_goal_angle - laser_ori[2] + np.pi) % (2 * np.pi) - np.pi  # Diferença de ângulo entre a orientação do robô e a direção para o objetivo

        if np.abs(angle_difference) > np.pi / 2:  # Se o ângulo de diferença for maior que 90 graus
            # Girar o robô em torno de seu eixo
            v = 0
            w = np.sign(angle_difference) * maxw
        else:
            # Calcular as velocidades das rodas conforme a Formula de Luca e Oriolo para calcular as velocidades
            v = (kv * ((Ft_x*(np.cos(laser_ori[2])) + Ft_y*(np.sin(laser_ori[2])))))
            w = (kw*((np.arctan2(Ft_y, Ft_x))-laser_ori[2]))
            w = (w + np.pi) % (2 * np.pi) - np.pi 

            # limit v,w to +/- max
            v = max(min(v, maxv), -maxv)
            w = max(min(w, maxw), -maxw)

        # Calcular as velocidades das rodas
        vr = (2*v + L*w) / (2*r)
        vl = (2*v - L*w) / (2*r)

        # Setando as velocidades
        sim.setJointTargetVelocity(motorLeft, 0.5*vl)
        sim.setJointTargetVelocity(motorRight, 0.5*vr)

        # se o robô estiver a certa distância do goal, parar e sair do loop
        if np.linalg.norm(np.array(laser_robot_position[:2]) - np.array(goal[:2])) < 1.5:
            sim.setJointTargetVelocity(motorLeft, 0)
            sim.setJointTargetVelocity(motorRight, 0)
            print("Chegou ao objetivo!")
            break

else:
    print("Failed to connect to remote API server")
    print("Exiting...")

# Stop the simulation:
sim.stopSimulation()

print("Program ended")

Connected to remote API server
laser handle ->  54
Posição do robô:  [7.5555792061951035, -8.475138551693885, 0.13864175602005718]
Ft new ->  [-0.70555792  1.32251386]
Força total ->  [-0.70555792  1.32251386]
Posição do robô:  [7.559510538153343, -8.482139664976685, 0.13866259376468146]
Ft new ->  [-0.70595105  1.32321397]
Força total ->  [-0.70595105  1.32321397]
Posição do robô:  [7.56381385754341, -8.48739426015155, 0.13866928449359853]
Ft new ->  [-0.70638139  1.32373943]
Força total ->  [-0.70638139  1.32373943]
Posição do robô:  [7.568744270633286, -8.491920477513414, 0.13867087735600706]
Ft new ->  [-0.70687443  1.32419205]
Força total ->  [-0.70687443  1.32419205]
Posição do robô:  [7.576306701463035, -8.495689268665185, 0.13865683713560162]
Ft new ->  [-0.70763067  1.32456893]
Força total ->  [-0.70763067  1.32456893]
Posição do robô:  [7.5946421156449, -8.487410578543889, 0.13866225201140073]
Ft new ->  [-0.70946421  1.32374106]
Força total ->  [-0.70946421  1.32374106]
Posi